#Download Data
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()
api.dataset_download_files('navoneel/brain-mri-images-for-brain-tumor-detection')

from zipfile import ZipFile

with ZipFile('brain-mri-images-for-brain-tumor-detection.zip', 'r') as myzip:
    myzip.extractall('brain-mri-images')

In [32]:
#Imports
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import classification_report, confusion_matrix

In [9]:
#Function to load images
def load_resize_images(filepath, target_size=(150,150)):
    images = []
    for file in os.listdir(filepath):
        image_path = os.path.join(filepath, file)
        with Image.open(image_path) as img:
            img = img.convert('RGB')
            img = img.resize(target_size)
            images.append(np.array(img))
    return images

In [10]:
positive_images = load_resize_images('brain-mri-images/yes')
negative_images = load_resize_images('brain-mri-images/no')

In [11]:
positive_labels = [1] * len(positive_images)
negative_labels = [0] * len(negative_images)

images = np.array(positive_images + negative_images)
labels = np.array(positive_labels + negative_labels)

images, labels = shuffle(images, labels, random_state=42)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=.2, random_state=42)

In [16]:
X_train = X_train/255.0
X_test = X_test/255.0

In [28]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [29]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [30]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=.2)

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 176ms/step - accuracy: 0.5515 - loss: 0.9116 - val_accuracy: 0.6341 - val_loss: 1.0464
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 156ms/step - accuracy: 0.6241 - loss: 0.8562 - val_accuracy: 0.7073 - val_loss: 0.5751
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step - accuracy: 0.8412 - loss: 0.4432 - val_accuracy: 0.7073 - val_loss: 0.6263
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 156ms/step - accuracy: 0.8682 - loss: 0.3612 - val_accuracy: 0.6341 - val_loss: 0.6831
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step - accuracy: 0.8527 - loss: 0.3497 - val_accuracy: 0.7317 - val_loss: 0.5788
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step - accuracy: 0.9351 - loss: 0.2510 - val_accuracy: 0.7073 - val_loss: 0.8247
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step - accuracy: 0.8876 - loss: 0.2527 - val_accuracy: 0.7317 - val_loss: 0.6854
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 151ms/step - accuracy: 0.9935 - loss: 0.1079 - val_accuracy: 0.7805 - val_loss:

In [31]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8695 - loss: 0.6941
Test Accuracy: 0.8823529481887817


In [33]:
predictions = model.predict(X_test)
y_pred = (predictions > 0.5).astype(int)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
[[13  6]
 [ 0 32]]
              precision    recall  f1-score   support

           0       1.00      0.68      0.81        19
           1       0.84      1.00      0.91        32

    accuracy                           0.88        51
   macro avg       0.92      0.84      0.86        51
weighted avg       0.90      0.88      0.88        51

